In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [36]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
print(df.shape)
df.head()

(50882, 14)


,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3,1.0,16,10404.0,0


In [37]:
df["Holding_Policy_Type"].fillna("0", inplace = True)
df["Holding_Policy_Duration"].fillna("0", inplace = True)
df["Holding_Policy_Duration"] = df["Holding_Policy_Duration"].replace(['14+'],'15') 
#df["Health Indicator"] = df["Health Indicator"].fillna("0", inplace = True)

df_test["Holding_Policy_Type"].fillna("0", inplace = True)
df_test["Holding_Policy_Duration"].fillna("0", inplace = True)
df_test["Holding_Policy_Duration"] = df_test["Holding_Policy_Duration"].replace(['14+'],'15')

In [38]:
df['City_Code'] = df['City_Code'].astype('category').cat.codes
df['Accomodation_Type'] = df['Accomodation_Type'].astype('category').cat.codes
df['Holding_Policy_Type'] = df['Holding_Policy_Type'].astype('category').cat.codes
df['Reco_Insurance_Type'] = df['Reco_Insurance_Type'].astype('category').cat.codes
df['Is_Spouse'] = df['Is_Spouse'].astype('category').cat.codes
df['Health Indicator'] = df['Health Indicator'].astype('category').cat.codes
df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].astype('category').cat.codes


df_test['City_Code'] = df_test['City_Code'].astype('category').cat.codes
df_test['Accomodation_Type'] = df_test['Accomodation_Type'].astype('category').cat.codes
df_test['Holding_Policy_Type'] = df_test['Holding_Policy_Type'].astype('category').cat.codes
df_test['Reco_Insurance_Type'] = df_test['Reco_Insurance_Type'].astype('category').cat.codes
df_test['Is_Spouse'] = df_test['Is_Spouse'].astype('category').cat.codes
df_test['Health Indicator'] = df_test['Health Indicator'].astype('category').cat.codes
df_test['Holding_Policy_Duration'] = df_test['Holding_Policy_Duration'].astype('category').cat.codes

In [39]:
df.corr()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
ID,1.000000,-0.003275,-0.000465,-0.004649,-0.008505,-0.001725,0.001101,-0.004262,0.002255,0.001933,0.002073,-0.002235,-0.002350,0.005159
City_Code,-0.003275,1.000000,0.059562,0.054010,-0.053742,-0.144504,-0.136441,-0.052259,-0.010050,-0.048641,0.025624,-0.084928,-0.144310,-0.000495
Region_Code,-0.000465,0.059562,1.000000,0.009105,-0.004275,-0.005649,-0.005928,-0.002567,0.014765,0.006604,0.006901,-0.065120,-0.010797,0.001121
Accomodation_Type,-0.004649,0.054010,0.009105,1.000000,-0.196833,-0.419129,-0.369400,-0.191045,-0.031541,-0.155073,0.076004,-0.022055,-0.350514,-0.005201
Reco_Insurance_Type,-0.008505,-0.053742,-0.004275,-0.196833,1.000000,0.227119,-0.014998,0.881560,0.035132,0.077088,-0.040672,0.020826,0.596160,0.009891
Upper_Age,-0.001725,-0.144504,-0.005649,-0.419129,0.227119,1.000000,0.921392,0.202947,0.015714,0.251589,-0.187927,0.025257,0.792689,0.002772
Lower_Age,0.001101,-0.136441,-0.005928,-0.369400,-0.014998,0.921392,1.000000,0.062634,0.011100,0.229052,-0.172292,0.021163,0.615739,-0.002099
Is_Spouse,-0.004262,-0.052259,-0.002567,-0.191045,0.881560,0.202947,0.062634,1.000000,0.037621,0.070357,-0.028768,0.022676,0.514727,0.003859
Health Indicator,0.002255,-0.010050,0.014765,-0.031541,0.035132,0.015714,0.011100,0.037621,1.000000,0.006835,0.008167,-0.002155,0.029464,0.003824
Holding_Policy_Duration,0.001933,-0.048641,0.006604,-0.155073,0.077088,0.251589,0.229052,0.070357,0.006835,1.000000,-0.637413,0.023474,0.194254,0.005581


In [40]:
#X = df.drop(['ID','Accomodation_Type','Health Indicator','Response'], axis = 1)
X = df.drop(['ID','City_Code','Region_Code','Is_Spouse','Holding_Policy_Duration','Holding_Policy_Type','Health Indicator','Reco_Policy_Premium','Response'], axis = 1)
#X = df.drop(['ID','Response'], axis = 1)
targets = df['Response'].values

#X_test = df_test.drop(['ID','Accomodation_Type','Health Indicator'], axis = 1)

X_test = df_test.drop(['ID','City_Code','Region_Code','Is_Spouse','Holding_Policy_Duration','Holding_Policy_Type','Health Indicator','Reco_Policy_Premium'], axis = 1)

In [41]:
X

,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Reco_Policy_Cat
0,1,0,36,36,22
1,0,1,75,22,22
2,0,0,32,32,19
3,0,1,52,48,19
4,1,0,44,44,16
...,...,...,...,...,...
50877,1,0,22,22,18
50878,1,0,27,27,4
50879,1,0,63,63,12
50880,0,1,71,49,16


In [42]:
X_test

,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Reco_Policy_Cat
0,0,0,30,30,5
1,0,1,69,68,18
2,1,0,28,28,17
3,1,0,23,23,18
4,0,0,75,75,5
...,...,...,...,...,...
21800,0,0,45,45,18
21801,0,0,59,59,18
21802,0,0,74,74,1
21803,1,0,25,25,19


In [43]:
X = preprocessing.normalize(X)
X_test = preprocessing.normalize(X_test)

In [44]:
#feature Scaling  

scaler = RobustScaler() 

X = scaler.fit_transform(X) 
X_test = scaler.fit_transform(X_test) 


In [45]:
#Bagging

kf = StratifiedKFold(n_splits=5, random_state=2018)
k = 0

# Keep some things after each loop
val_auc_scores =  []
val_preds = []
val_true = []
for train_idx, val_idx in kf.split(X, targets):
#     Select the dat for this fold
    X_train, y_train = X[train_idx], targets[train_idx]
    X_val, y_val = X[val_idx], targets[val_idx]
    
#     Create simple model
    
    model = RandomForestClassifier()
    bags = 10
    seed = 1

    bagged_prediction = np.zeros(X_val.shape[0])

    for n in range (0, bags):
        model.set_params(random_state = seed + n)
        model.fit(X_train, y_train)
        #val_probs = model.predict(test)
        val_probs = model.predict_proba(X_val)[:, 1]
        train_probs = model.predict_proba(X_train)[:, 1]
        bagged_prediction += val_probs

    bagged_prediction /= bags
    

#     Calculate ROC AUC
    train_auc = roc_auc_score(y_train, train_probs)
    val_auc = roc_auc_score(y_val, val_probs)
    
#     Keep track of val auc
    val_auc_scores.append(val_auc)
    
    print(f'Fold: {k}')
    print(f'\tTrain AUC: {train_auc}\n\tVal AUC: {val_auc} ')
    
    k += 1
    
    val_preds.extend(model.predict(X_val))
    val_true.extend(y_val)
    
print(f'\nMean Validation ROC AUC: {np.mean(val_auc_scores)}')

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


Fold: 0
	Train AUC: 0.7898928709498249
	Val AUC: 0.5872246989894049 
Fold: 1
	Train AUC: 0.7885911430559194
	Val AUC: 0.595814439932087 
Fold: 2
	Train AUC: 0.7910115378176501
	Val AUC: 0.5803292810522249 
Fold: 3
	Train AUC: 0.7913544512775518
	Val AUC: 0.5852378226311508 
Fold: 4
	Train AUC: 0.7923145655650008
	Val AUC: 0.5806063751176241 

Mean Validation ROC AUC: 0.5858425235444984


In [46]:
cm = pd.DataFrame(confusion_matrix(val_true, val_preds, labels=[0,1]))
cm.columns = ['pred_Zero','pred_One']
cm.index = ['Zero','One']

In [47]:
cm

,pred_Zero,pred_One
Zero,36896,1777
One,11365,844


In [48]:
#Predicting the test set result  
op = model.predict(X_test)  

In [50]:
ID = df_test['ID']

In [51]:
g = {
    'ID' : [],
    'Response' : []
}

In [52]:
for i in range(len(ID)):
    g['ID'].append(ID[i])
    g['Response'].append(op[i])

In [53]:
ab = pd.DataFrame(g)

In [54]:
ab.to_csv('response.csv', index = False)